In [1]:
from rich import print as rprint

In [2]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("Testing")

<Experiment: artifact_location='mlflow-artifacts:/785514504126471133', creation_time=1755920871238, experiment_id='785514504126471133', last_update_time=1755920871238, lifecycle_stage='active', name='Testing', tags={}>

### Basic LLM call


#### Sync


In [3]:
from tinyloop.modules.generate import Generate
from pydantic import BaseModel
from typing import List


class Character(BaseModel):
    name: str
    description: str
    image: str


class Characters(BaseModel):
    characters: List[Character]


generate = Generate(
    model="openai/gpt-4.1-nano", temperature=0.1, output_format=Characters
)

response = generate(prompt="Give me 3 harry potter characters")
rprint(response)

LLMResponse(
    response=Characters(
        characters=[
            Character(
                name='Harry Potter',
                description="The main protagonist, known as the Boy Who Lived, a young wizard famous for surviving 
Lord Voldemort's curse as a baby.",
                image='https://example.com/images/harry_potter.jpg'
            ),
            Character(
                name='Hermione Granger',
                description="Harry's intelligent and resourceful friend, a Muggle-born witch known for her academic
excellence and bravery.",
                image='https://example.com/images/hermione_granger.jpg'
            ),
            Character(
                name='Ron Weasley',
                description="Harry's loyal friend from a large wizarding family, known for his humor and bravery.",
                image='https://example.com/images/ron_weasley.jpg'
            )
        ]
    ),
    raw_response=ModelResponse(
        id='chatcmpl-C7aABNeGpiLRRaoiVimuVTYLEMbLM',
        created=1755923343,
        model='gpt-4.1-nano-2025-04-14',
        object='chat.completion',
        system_fingerprint='fp_c4c155951e',
        choices=[
            Choices(
                finish_reason='stop',
                index=0,
                message=Message(
                    content='{"characters":[{"name":"Harry Potter","description":"The main protagonist, known as 
the Boy Who Lived, a young wizard famous for surviving Lord Voldemort\'s curse as a 
baby.","image":"https://example.com/images/harry_potter.jpg"},{"name":"Hermione Granger","description":"Harry\'s 
intelligent and resourceful friend, a Muggle-born witch known for her academic excellence and 
bravery.","image":"https://example.com/images/hermione_granger.jpg"},{"name":"Ron Weasley","description":"Harry\'s 
loyal friend from a large wizarding family, known for his humor and 
bravery.","image":"https://example.com/images/ron_weasley.jpg"}]}',
                    role='assistant',
                    tool_calls=None,
                    function_call=None,
                    provider_specific_fields={'refusal': None},
                    annotations=[]
                ),
                provider_specific_fields={}
            )
        ],
        usage=Usage(
            completion_tokens=138,
            prompt_tokens=113,
            total_tokens=251,
            completion_tokens_details=CompletionTokensDetailsWrapper(
                accepted_prediction_tokens=0,
                audio_tokens=0,
                reasoning_tokens=0,
                rejected_prediction_tokens=0,
                text_tokens=None
            ),
            prompt_tokens_details=PromptTokensDetailsWrapper(
                audio_tokens=0,
                cached_tokens=0,
                text_tokens=None,
                image_tokens=None
            )
        ),
        service_tier='default'
    ),
    tool_calls=None,
    message_history=[
        {'role': 'user', 'content': 'Give me 3 harry potter characters'},
        {
            'role': 'assistant',
            'content': '{"characters":[{"name":"Harry Potter","description":"The main protagonist, known as the Boy
Who Lived, a young wizard famous for surviving Lord Voldemort\'s curse as a 
baby.","image":"https://example.com/images/harry_potter.jpg"},{"name":"Hermione Granger","description":"Harry\'s 
intelligent and resourceful friend, a Muggle-born witch known for her academic excellence and 
bravery.","image":"https://example.com/images/hermione_granger.jpg"},{"name":"Ron Weasley","description":"Harry\'s 
loyal friend from a large wizarding family, known for his humor and 
bravery.","image":"https://example.com/images/ron_weasley.jpg"}]}'
        }
    ],
    cost=6.65e-05,
    hidden_fields={
        'custom_llm_provider': 'openai',
        'region_name': None,
        'additional_headers': {
            'x-ratelimit-limit-requests': '10000',
            'x-ratelimit-remaining-requests': '9999',
            'x-ratel

Trace(trace_id=tr-a7981ae12e903bb87aae8b7124460b88)

In [ ]:
from tinyloop.inference.litellm import LLM
import json
from tinyloop.features.function_calling import Tool


def get_current_weather(location: str, unit: str):
    """Get the current weather in a given location

    Parameters
    ----------
    location : str
        The city and state, e.g. San Francisco, CA
    unit : str {'celsius', 'fahrenheit'}
        Temperature unit

    Returns
    -------
    str
        a sentence indicating the weather
    """
    if location == "Boston, MA":
        return "The weather is 12F"
    return f"Weather in {location} is sunny"


llm = LLM(
    model="openai/gpt-4.1-nano",
    temperature=0.1,
)

weather_tool = Tool(get_current_weather)

inference = llm(
    prompt="What is the weather in Boston, MA?",
    tools=[weather_tool],
)

for tool_call in inference.raw_response.choices[0].message.tool_calls:
    tool_name = tool_call.function.name
    tool_args = json.loads(tool_call.function.arguments)
    print(f"Tool: {tool_name}")
    print(f"Args: {tool_args}")
    print(weather_tool(**tool_args))

In [ ]:
rprint(generate(prompt="Give me 3 differentharry potter characters"))

In [ ]:
generate.llm.get_history()

#### Async


In [ ]:
from tinyloop.inference.litellm import LLM

llm = LLM(model="openai/gpt-3.5-turbo", temperature=0.1)

response = await llm.acall(prompt="Hello, how are you?")
rprint(response)

In [ ]:
llm.get_history()